Co należy zrobić w zadaniu?

Połącz się z nowym API, które dostarczyliśmy. Komunikujesz się z nim przez JSON-a

Zdobądź strukturę tabel, które Cię interesują

Przekaż strukturę do LLM-a i poproś o przygotowanie zapytania SQL, które spełnia nasze wymagania

Zdobądź odpowiedź na pytanie: “które aktywne datacenter (DC_ID) są zarządzane przez pracowników, którzy są na urlopie (is_active=0)”

Odpowiedź (w formie tablicy) prześlij w standardowy sposób do centrali jako zadanie database.

Jeśli odpowiedź zostanie potwierdzona przez centralę, otrzymasz flagę

Podpowiedź: to zadanie możesz wykonać na wiele sposobów, ale najbardziej rozwojowym byłoby sprawienie, aby wszystkie potrzebne informacje pozyskiwane były w sposób dynamiczny, bez hardcodowania informacji o strukturze bazy w Twoim programie.

In [14]:
import os
import requests
import json
from dotenv import load_dotenv

In [15]:
load_dotenv()

personal_api_key = os.getenv("PERSONAL_API_KEY")

In [42]:
querry_json_user = {
    "task": "database",
    "apikey": personal_api_key,
    "query": "select * from users limit 1"
}
get_data_users = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_user)
print(get_data_users.json())

{'reply': [{'id': '1', 'username': 'Adrian', 'access_level': 'user', 'is_active': '1', 'lastlog': '2023-06-12'}], 'error': 'OK'}


In [43]:
querry_json_tables = {
    "task": "database",
    "apikey": personal_api_key,
    "query": "show tables"
}
get_data_tables = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_tables)
print(get_data_tables.json())

{'reply': [{'Tables_in_banan': 'connections'}, {'Tables_in_banan': 'correct_order'}, {'Tables_in_banan': 'datacenters'}, {'Tables_in_banan': 'users'}], 'error': 'OK'}


In [67]:
get_data_tables_struct = {}
for table in get_data_tables.json()["reply"]:    
    print(table["Tables_in_banan"])
    querry_json_tables_struct = {
        "task": "database",
        "apikey": personal_api_key,
        "query": f"show create table {table["Tables_in_banan"]}"
    }
    get_data_tables_struct[table["Tables_in_banan"]] = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_tables_struct).json()
print(get_data_tables_struct)


connections
correct_order
datacenters
users
{'connections': {'reply': [{'Table': 'connections', 'Create Table': 'CREATE TABLE `connections` (\n  `user1_id` int(11) NOT NULL,\n  `user2_id` int(11) NOT NULL,\n  PRIMARY KEY (`user1_id`,`user2_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci'}], 'error': 'OK'}, 'correct_order': {'reply': [{'Table': 'correct_order', 'Create Table': 'CREATE TABLE `correct_order` (\n  `base_id` int(11) DEFAULT NULL,\n  `letter` char(1) DEFAULT NULL,\n  `weight` int(11) DEFAULT 0\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci'}], 'error': 'OK'}, 'datacenters': {'reply': [{'Table': 'datacenters', 'Create Table': 'CREATE TABLE `datacenters` (\n  `dc_id` int(11) DEFAULT NULL,\n  `location` varchar(30) NOT NULL,\n  `manager` int(11) NOT NULL DEFAULT 31,\n  `is_active` int(11) DEFAULT 0\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci'}], 'error': 'OK'}, 'users': {'reply': [{'Table': 'users', 'Create T

In [68]:
querry_json_users_data = {
    "task": "database",
    "apikey": personal_api_key,
    "query": "select username, access_level, is_active from users"
}
get_data_users_data = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_users_data)
print(json.dumps(get_data_users_data.json(), indent=4, ensure_ascii=False))

{
    "reply": [
        {
            "username": "Adrian",
            "access_level": "user",
            "is_active": "1"
        },
        {
            "username": "Monika",
            "access_level": "user",
            "is_active": "1"
        },
        {
            "username": "Azazel",
            "access_level": "removed",
            "is_active": "0"
        },
        {
            "username": "Robert",
            "access_level": "user",
            "is_active": "1"
        },
        {
            "username": "Aleksandra",
            "access_level": "user",
            "is_active": "1"
        },
        {
            "username": "Michał",
            "access_level": "user",
            "is_active": "1"
        },
        {
            "username": "Katarzyna",
            "access_level": "user",
            "is_active": "1"
        },
        {
            "username": "Mateusz",
            "access_level": "user",
            "is_active": "1"
        },
        {
  

In [69]:
querry_json_datacenter_data = {
    "task": "database",
    "apikey": personal_api_key,
    "query": "select * from datacenters"
}
get_data_datacenters_data = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_datacenter_data)
print(json.dumps(get_data_datacenters_data.json(), indent=4, ensure_ascii=False))


{
    "reply": [
        {
            "dc_id": "8547",
            "location": "Warszawa",
            "manager": "22",
            "is_active": "0"
        },
        {
            "dc_id": "1226",
            "location": "Kraków",
            "manager": "44",
            "is_active": "1"
        },
        {
            "dc_id": "6491",
            "location": "Wrocław",
            "manager": "31",
            "is_active": "1"
        },
        {
            "dc_id": "9775",
            "location": "Poznań",
            "manager": "69",
            "is_active": "0"
        },
        {
            "dc_id": "1405",
            "location": "Łódź",
            "manager": "13",
            "is_active": "1"
        },
        {
            "dc_id": "3698",
            "location": "Działdowo",
            "manager": "33",
            "is_active": "0"
        },
        {
            "dc_id": "4278",
            "location": "Gdańsk",
            "manager": "28",
            "is_active": 

Mogę spróbować dalej pracować sam, ale w zadaniu jest informacja o wykorzystaniu llm po pobraniu struktury 

In [70]:
import sys
sys.path.append('/home/pempek18/Desktop/AI_DEVS')  # path to the directory containing ai_devs folder
from ai_devs.OpenAIService import OpenAIService
openai = OpenAIService()

In [72]:
query = "które aktywne datacenter (DC_ID) są zarządzane przez pracowników, którzy są na urlopie (is_active=0)"
response = openai.completion(messages=[
      {
        "role": "system",
        "content": f'''You are a helpful assistant that helps to find answer in database.
                  You are given a query and a structure of database.
                  You need to prepare a SQL query that will answer the question.
                  datacenter struct : {get_data_tables_struct}.
                  answer only with SQL query
                  think twice if the name of the table is correct'''
      },
      { "role": "user", "content": query }])
print(response.choices[0].message.content)

```sql
SELECT d.dc_id
FROM datacenters d
JOIN users u ON d.manager = u.id
WHERE d.is_active = 1 AND u.is_active = 0;
```


In [73]:
querry_json_final = {
    "task": "database",
    "apikey": personal_api_key,
    "query": response.choices[0].message.content.replace("```sql", "").replace("```", "")
}
get_data_final = requests.post("https://centrala.ag3nts.org/apidb", json=querry_json_final)
print(json.dumps(get_data_final.json(), indent=4, ensure_ascii=False))

{
    "reply": [
        {
            "dc_id": "4278"
        },
        {
            "dc_id": "9294"
        }
    ],
    "error": "OK"
}


In [78]:
answer = [get_data_final.json()["reply"][0]["dc_id"], get_data_final.json()["reply"][1]["dc_id"]]
answer_json = {
    "task": "database",
    "apikey": personal_api_key,
    "answer": answer
}
answer_url = "https://centrala.ag3nts.org/report"
answer_response = requests.post(answer_url, json=answer_json)
print(answer_response.request.body)  # Displays the request body

# Assuming `login_response` is the response object from the login request
print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
print("Response Body:", answer_response.text.encode('utf-8'))       # Displays the response content as a string    

b'{"task": "database", "apikey": "1400cbf0-b7dd-49ab-9342-6ad8fd26ba69", "answer": ["4278", "9294"]}'
Status Code: 200
Response Body: b'{\n    "code": 0,\n    "message": "{{FLG:KNOWLEDGE}}"\n}'
